# Simple Test Class


In [117]:
%%writefile testsfolder/my_tests.py

import unittest

class MyTests(unittest.TestCase):
    def test_something(self):
        self.assertEqual(1, 1)
    def test_something_else(self):
        self.assertEqual(0, 0)

Overwriting testsfolder/my_tests.py


# Simple Execution


## Specific File


In [118]:
!python3 -m unittest testsfolder/my_tests.py

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## Module

Note the following:

- no slashes (dots instead)
- no file extension


In [119]:
!python3 -m unittest testsfolder.my_tests

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## Specific Class

To do this, you have to load as a module instead of a file.


In [120]:
!python3 -m unittest testsfolder.my_tests.MyTests

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## Specific Method

To do this, you have to:

- load as a module
- specify the classname as well


In [121]:
!python3 -m unittest testsfolder.my_tests.MyTests.test_something

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


# Printing from Within Test


In [122]:
%%writefile testsfolder/mytests_with_output.py

import unittest

class MyTests(unittest.TestCase):
    def test_something(self):
        x = 0
        
        print(f'ATTENTION: Value of x before test is {x}')
        self.assertEqual(1, 1)

Overwriting testsfolder/mytests_with_output.py


In [123]:
!python3 -m unittest testsfolder.mytests_with_output

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ATTENTION: Value of x before test is 0
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


# Test Class Members

You will often want common helper methods and variables to capture side effects in tests. It is safe to put these on the test class as long as they are **instance members** and not static members. Each test method gets executed **in its own class instance**, so as long as nothing is static, they won't interfere with each other.

Also note that to create instance variables, you need to **override \_\_init\_\_** correctly as shown.

Also note that the test framework uses the **test\_ prefix** to tell which methods are tests.


In [124]:
%%writefile testsfolder/mytests_with_members.py

import unittest

# Pretend this was imported from an API we will test.
class Transmitter:
    def transmit_x(self, fn):
        fn(10)
        
class MyTests(unittest.TestCase):
    def __init__(self, methodName='runTest'):
        super().__init__(methodName)
        
        # Instance variable
        self.captured_x = None
        
    # Helper method
    def capture_x(self, x):
        self.captured_x = x
        
    def test_1(self):
        transmitter = Transmitter()
        
        transmitter.transmit_x(self.capture_x)
        
        self.assertEqual(self.captured_x, 10)
        
    # If they ran in the same instance, this would fail.
    def test_2(self):
        self.assertIsNone(self.captured_x)

Overwriting testsfolder/mytests_with_members.py


In [125]:
!python3 -m unittest testsfolder.mytests_with_members

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


# Yoda Conditions

Some people prefer to write `self.assertEqual(10, x)` instead of `self.assertEqual(x, 10)`. For a lot of asserts like assertEqual, it doesn't matter. However, some asserts, especially the ones in **tensorflow.test.TestCase**, actually make less sense.

Overall, pay attention to the **param names** of assert methods to make sure you're using them as intended. For instance, if you used the non-yoda version for tensorflow.test.TestCase.assertAllEqual, the output would say "expected x but got 10" which would be confusing when debugging.


# Some Useful Asserts

There are a lot of asserts you can use in the TestCase class. Here are just a few examples.


In [126]:
%%writefile testsfolder/my_tests_asserts.py

import unittest

class MyTests(unittest.TestCase):
    def test_int(self):
        x = 0
        self.assertEqual(x, 0)
        
    def test_string(self):
        x = 'hi'
        self.assertEqual(x, 'hi')
        
    def test_none(self):
        x = None
        self.assertIsNone(x)
        
    def test_negation(self):
        self.assertIsNotNone(10)
        ## Use of Is is not consistent in the asserts.
        self.assertNotEqual(10, 11)
        
    def test_list(self):
        x = [1, 2, 3]
        self.assertListEqual(x, [1, 2, 3])
        
    def test_dict(self):
        x = {'a': 'hi'}
        self.assertDictEqual(x, {'a': 'hi'})
        
    def test_type(self):
        x = 5
        self.assertIsInstance(x, int)
        
    def test_reference(self):
        x = [1, 2, 3]
        self.assertIs(x, x)
        self.assertIsNot(x, [1, 2, 3])

Overwriting testsfolder/my_tests_asserts.py


In [127]:
!python3 -m unittest testsfolder.my_tests_asserts

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
........
----------------------------------------------------------------------
Ran 8 tests in 0.000s

OK


# Tensorflow

To assert between tensors and numpy arrays, python arrays, etc. you just need to derive from a **different base** to get the extra assert methods.

Unfortunately, you will also see the multitude of irrelevant warnings your TF prints if you haven't turned them off.


In [128]:
%%writefile testsfolder/my_tests_tf.py

import tensorflow as tf

class MyTests(tf.test.TestCase):
    def test_tensor(self):
        x = tf.constant([[1, 2, 3], [4, 5, 6]])
        # This is a case where you have to do Yoda conditions.
        self.assertAllEqual([[1, 2, 3], [4, 5, 6]], x)

Overwriting testsfolder/my_tests_tf.py


In [129]:
!python3 -m unittest testsfolder.my_tests_tf

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-05-19 11:45:44.668931: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 11:45:45.258768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
s2023-05-19 11:45:46.019127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-19 11:45:46.044091: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have

# Mocking

The mocking library is very big and complex. Only a few examples are shown here.


## Mocking print()

Note below that prints that happen within the `with` block do not get printed to the console but instead get appended to our array. Once the block is over, printing goes to the console again.


In [130]:
%%writefile testsfolder/my_tests_mocking.py

import unittest
from unittest.mock import patch

def my_print(val):
    print(val)
    
class MyTests(unittest.TestCase):
    def test_my_print(self):
        printed = []
        with patch('builtins.print', new = lambda *args,**_: printed.append(args[0])):
            my_print(10)
            my_print('hi')
        print('ATTENTION: printed after mocking ended')
        self.assertListEqual(printed, [10, 'hi'])

Overwriting testsfolder/my_tests_mocking.py


In [131]:
!python3 -m unittest testsfolder.my_tests_mocking

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ATTENTION: printed after mocking ended
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## Mocking Other Stuff


In [132]:
%%writefile testsfolder/my_tests_mocking_general.py

import unittest
from unittest.mock import patch

from os import path
    
class MyTests(unittest.TestCase):
    def test_dirname(self):
        captured = []
        # Need to use fully qualified name of symbol instead of how you imported it.
        with patch('os.path.dirname', new = lambda file: captured.append(file)):
            path.dirname('some file')
        # This one won't append.
        path.dirname(__file__)
        self.assertListEqual(captured, ['some file'])

Overwriting testsfolder/my_tests_mocking_general.py


In [133]:
!python3 -m unittest testsfolder.my_tests_mocking_general

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## Mocking Return Value

You don't have to do a lambda.


In [134]:
%%writefile testsfolder/my_tests_mocking_return.py

import unittest
from unittest.mock import patch

from os import path
    
class MyTests(unittest.TestCase):
    def test_dirname(self):
        with patch('os.path.dirname') as mock_dirname:
            mock_dirname.return_value = 'hi there'
            self.assertEqual(path.dirname('anything'), 'hi there')

Overwriting testsfolder/my_tests_mocking_return.py


In [135]:
!python3 -m unittest testsfolder.my_tests_mocking_return

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## Fake

In this example, our class uses an expensive API that we do not want to actually use in our test. We can fake the API so that our class thinks it's using the real thing, and then test that our class did the right thing.


In [136]:
%%writefile testsfolder/my_tests_mocking_fake.py

import unittest
from unittest.mock import patch

# Assume this is in some API somewhere.
class SomeExpensiveApi:
    def __init__(self):
        self.x = 10
    def f(self):
        return self.x
    
def get_expensive_thing():
    return SomeExpensiveApi()

# Assume this is our own class we want to test.
class MyClass:
    def g(self):
        expensive = get_expensive_thing()
        return expensive.f()
    
class MyTests(unittest.TestCase):
    class MyFake:
        def __init__(self, val):
            self.val = val
        def f(self):
            return self.val
        
    def test_g(self):
        with patch(__name__ + '.get_expensive_thing', new = lambda: MyTests.MyFake(20)):
            m = MyClass()
            self.assertEqual(m.g(), 20)

Overwriting testsfolder/my_tests_mocking_fake.py


In [137]:
!python3 -m unittest testsfolder.my_tests_mocking_fake

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK
